In [37]:
import heapq

def state_check(state):
    """check the format of state, and return corresponding goal state.
       Do NOT edit this function."""
    non_zero_numbers = [n for n in state if n != 0]
    num_tiles = len(non_zero_numbers)
    if num_tiles == 0:
        raise ValueError('At least one number is not zero.')
    elif num_tiles > 9:
        raise ValueError('At most nine numbers in the state.')
    matched_seq = list(range(1, num_tiles + 1))
    if len(state) != 9 or not all(isinstance(n, int) for n in state):
        raise ValueError('State must be a list contain 9 integers.')
    elif not all(0 <= n <= 9 for n in state):
        raise ValueError('The number in state must be within [0,9].')
    elif len(set(non_zero_numbers)) != len(non_zero_numbers):
        raise ValueError('State can not have repeated numbers, except 0.')
    elif sorted(non_zero_numbers) != matched_seq:
        raise ValueError('For puzzles with X tiles, the non-zero numbers must be within [1,X], '
                          'and there will be 9-X grids labeled as 0.')
    goal_state = matched_seq
    for _ in range(9 - num_tiles):
        goal_state.append(0)
    return tuple(goal_state)

def get_manhattan_distance(from_state, to_state):
    """
    TODO: implement this function. This function will not be tested directly by the grader. 

    INPUT: 
        Two states (The first one is current state, and the second one is goal state)

    RETURNS:
        A scalar that is the sum of Manhattan distances for all tiles.
    """
    distance = 0
    for i, val in enumerate(from_state):
        if val == 0:
            continue
        goal_index = to_state.index(val)
        x1, y1 = divmod(i, 3)
        x2, y2 = divmod(goal_index, 3)
        distance += abs(x1 - x2) + abs(y1 - y2)
    return distance

def naive_heuristic(from_state, to_state):
    """
    TODO: implement this function. This function will not be tested directly by the grader. 

    INPUT: 
        Two states (The first one is current state, and the second one is goal state)

    RETURNS:
        0 (but experimenting with other constants is encouraged)
    """
    return 0

def sum_of_squares_distance(from_state, to_state):
    """
    TODO: implement this function. This function will not be tested directly by the grader. 

    INPUT: 
        Two states (The first one is current state, and the second one is goal state)

    RETURNS:
        A scalar that is the sum of squared distances for all tiles
    """
    distance = 0
    for val in range(1, 9):
        if val in from_state:
            i1 = from_state.index(val)
            i2 = to_state.index(val)
            x1, y1 = divmod(i1, 3)
            x2, y2 = divmod(i2, 3)
            distance += (x1 - x2)**2 + (y1 - y2)**2
    return distance

def print_succ(state, heuristic=get_manhattan_distance):
    """
    TODO: This is based on get_succ function below, so should implement that function.

    INPUT: 
        A state (list of length 9)

    WHAT IT DOES:
        Prints the list of all the valid successors in the puzzle. 
    """

    # given state, check state format and get goal_state.
    goal_state = state_check(state)
    # please remove debugging prints when you submit your code.
    # print('initial state: ', state)
    # print('goal state: ', goal_state)

    succ_states = get_succ(state)

    for succ_state in succ_states:
        print(succ_state, "h={}".format(heuristic(succ_state,goal_state)))


def get_succ(state):
    """
    TODO: implement this function.

    INPUT: 
        A state (list of length 9)

    RETURNS:
        A list of all the valid successors in the puzzle (don't forget to sort the result as done below). 
    """
    succ_states = []
    zero_indices = [i for i, val in enumerate(state) if val == 0]

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # up, down, left, right

    for z in zero_indices:
        x, y = divmod(z, 3)
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 3 and 0 <= ny < 3:
                neighbor_idx = nx * 3 + ny
                if state[neighbor_idx] != 0:  # only move a non-zero tile into the 0
                    new_state = state.copy()
                    new_state[z], new_state[neighbor_idx] = new_state[neighbor_idx], new_state[z]
                    succ_states.append(new_state)

    return sorted(succ_states)

def is_solvable(state):
    tiles = [x for x in state if x != 0]
    inv_count = 0
    for i in range(len(tiles)):
        for j in range(i + 1, len(tiles)):
            if tiles[i] > tiles[j]:
                inv_count += 1
    return inv_count % 2 == 0

def solve(state, goal_state=[1, 2, 3, 4, 5, 6, 7, 0, 0], heuristic=get_manhattan_distance):
    """
    TODO: Implement the A* algorithm here.

    INPUT: 
        An initial state (list of length 9)

    WHAT IT SHOULD DO:
        Prints a path of configurations from initial state to goal state along  h values, number of moves, and max queue number in the format specified in the pdf.
    """

    # This is a format helper，which is only designed for format purpose.
    # define "solvable_condition" to check if the puzzle is really solvable
    # build "state_info_list", for each "state_info" in the list, it contains "current_state", "h" and "move".
    # define and compute "max_length", it might be useful in debugging
    # it can help to avoid any potential format issue.

    goal_state = list(state_check(state))
    visited = set()
    pq = []
    state_list = [state]
    parent = {-1: None}
    meta = {-1: (0, heuristic(state, goal_state))}

    start_h = heuristic(state, goal_state)
    heapq.heappush(pq, (start_h, state[:], (0, start_h, -1)))

    max_length = 1

    while pq:
        max_length = max(max_length, len(pq))
        f, curr_state, (g, h, parent_idx) = heapq.heappop(pq)
        curr_tup = tuple(curr_state)

        if curr_tup in visited:
            continue
        visited.add(curr_tup)

        curr_index = len(state_list)
        state_list.append(curr_state)
        parent[curr_index - 1] = parent_idx
        meta[curr_index - 1] = (g, h)

        if curr_state == goal_state:
            path = []
            idx = curr_index - 1
            while idx != -1:
                s = state_list[idx]
                g_val, h_val = meta[idx]
                path.append((s, h_val, g_val))
                idx = parent[idx]
            path.reverse()
            print("True")
            for s, h_val, g_val in path:
                print(f"{s} h={h_val} moves: {g_val}")
            print(f"Max queue length: {max_length}")
            return

        successors = get_succ(curr_state)
        for succ in successors:
            if tuple(succ) in visited:
                continue
            g_new = g + 1
            h_new = heuristic(succ, goal_state)
            f_new = g_new + h_new
            heapq.heappush(pq, (f_new, succ[:], (g_new, h_new, curr_index - 1)))

    
if __name__ == "__main__":
    """
    Feel free to write your own test code here to exaime the correctness of your functions. 
    Note that this part will not be graded.
    """
    #print_succ([2,5,1,4,0,6,7,0,3])
    # print()
    #
    # print(get_manhattan_distance([2,5,1,4,0,6,7,0,3], [1, 2, 3, 4, 5, 6, 7, 0, 0]))
    # print()

    solve([4,3,0,5,1,6,7,2,0], heuristic=get_manhattan_distance)
    print()


True
[4, 3, 0, 5, 1, 6, 7, 2, 0] h=7 moves: 0
[4, 3, 0, 5, 1, 6, 7, 2, 0] h=6 moves: 1
[4, 0, 3, 5, 1, 6, 7, 2, 0] h=5 moves: 2
[4, 1, 3, 5, 0, 6, 7, 2, 0] h=4 moves: 3
[4, 1, 3, 0, 5, 6, 7, 2, 0] h=3 moves: 4
[4, 1, 3, 5, 2, 6, 7, 0, 0] h=4 moves: 5
[0, 1, 0, 4, 5, 3, 7, 2, 6] h=5 moves: 6
[0, 1, 3, 4, 0, 6, 7, 5, 2] h=4 moves: 7
[1, 0, 3, 4, 0, 6, 7, 5, 2] h=3 moves: 8
[1, 0, 3, 4, 5, 6, 0, 7, 2] h=2 moves: 9
[1, 2, 3, 4, 0, 5, 7, 0, 6] h=1 moves: 10
[1, 2, 3, 4, 5, 0, 7, 0, 6] h=0 moves: 11
Max queue length: 91

